In [1]:
import ortools

In [2]:
from ortools.linear_solver import pywraplp


# Instantiate a mixed-integer solver, naming it SolveIntegerProblem.
solver = pywraplp.Solver('SolveIntegerProblem',
                       pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

In [3]:
result_status = solver.Solve()

In [4]:
x = solver.IntVar(0.0, 1.0, 'x') # red(x)
y = solver.IntVar(0.0, 1.0, 'y') # red(y)
z = solver.IntVar(0.0, 1.0, 'z') # blue(x)
w = solver.IntVar(0.0, 1.0, 'w') # blue(y)
xx = solver.IntVar(0.0, 1.0, 'xx') # red(x)
yy = solver.IntVar(0.0, 1.0, 'yy') # red(y)
zz = solver.IntVar(0.0, 1.0, 'zz') # blue(x)
ww = solver.IntVar(0.0, 1.0, 'ww') # blue(y)
px = 0.5
py = 0.6
pz = 0.3
pw = 0.3

In [5]:
constraintx = solver.Constraint(1., 1.)
constraintx.SetCoefficient(x, 1)
constraintx.SetCoefficient(xx, 1)

constrainty = solver.Constraint(1., 1.)
constrainty.SetCoefficient(y, 1)
constrainty.SetCoefficient(yy, 1)

constraintz = solver.Constraint(1., 1.)
constraintz.SetCoefficient(z, 1)
constraintz.SetCoefficient(zz, 1)

constraintw = solver.Constraint(1., 1.)
constraintw.SetCoefficient(w, 1)
constraintw.SetCoefficient(ww, 1)

In [6]:
# nr of red blocks >= nr of blue blocks
# == 0 >= nr blue - nr red == nr blue - nr red <= 0
constraint1 = solver.Constraint(-solver.infinity(), 0.0)
constraint1.SetCoefficient(x, -1)
constraint1.SetCoefficient(y, -1)
constraint1.SetCoefficient(z, 1)
constraint1.SetCoefficient(w, 1)

# x <= 3.5
#constraint2 = solver.Constraint(-solver.infinity(), 3.5)
#constraint2.SetCoefficient(x, 1)
#constraint2.SetCoefficient(y, 0)

In [7]:
# Maximise probability of the event
# Maximize x + 10 * y.
objective = solver.Objective()
objective.SetCoefficient(x, px)
objective.SetCoefficient(y, py)
objective.SetCoefficient(z, pz)
objective.SetCoefficient(w, pw)
objective = solver.Objective()
objective.SetCoefficient(xx, (1-px))
objective.SetCoefficient(yy, (1-py))
objective.SetCoefficient(zz, (1-pz))
objective.SetCoefficient(ww, (1-pw))
objective.SetMaximization()

In [8]:
result_status = solver.Solve()
# The problem has an optimal solution.
assert result_status == pywraplp.Solver.OPTIMAL

# The solution looks legit (when using solvers other than
# GLOP_LINEAR_PROGRAMMING, verifying the solution is highly recommended!).
assert solver.VerifySolution(1e-7, True)

print('Number of variables =', solver.NumVariables())
print('Number of constraints =', solver.NumConstraints())

# The objective value of the solution.
print('Optimal objective value = %d' % solver.Objective().Value())
print()
# The value of each variable in the solution.
variable_list = [x, y, z, w, xx, yy, zz, ww]

for variable in variable_list:
    print('%s = %d' % (variable.name(), variable.solution_value()))

Number of variables = 8
Number of constraints = 5
Optimal objective value = 2

x = 1
y = 1
z = 0
w = 0
xx = 0
yy = 0
zz = 1
ww = 1


How do we ensure, through a linear constraint, that the difference in the number of blue blocks and red block will not simply be equivalent to the same block being both red and blue

We could say that all blocks within the same rule should be different. But this is not great.
We could get all solutions, not just the best solution. Or is there a get top x solutions option?

In [14]:
from __future__ import print_function
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import solver_parameters_pb2

def main():
    # Instantiate a CP solver.
    parameters = pywrapcp.Solver.DefaultSolverParameters()
    solver = pywrapcp.Solver("simple_CP", parameters)

    # x and y are integer non-negative variables.
    x = solver.IntVar(0, 17, 'x')
    y = solver.IntVar(0, 17, 'y')
    solver.Add(2*x + (1-x)*2 + 14*y <= 35) # Notice how this allows x and (1-x) in the same constraint
    solver.Add(2*x  <= 7)
    obj_expr = solver.IntVar(0, 1000, "obj_expr")
    solver.Add(obj_expr == x + 10*y)
    objective = solver.Maximize(obj_expr, 1)
    decision_builder = solver.Phase([x, y],
                                  solver.CHOOSE_FIRST_UNBOUND,
                                  solver.ASSIGN_MIN_VALUE)
    # Create a solution collector.
    collector = solver.LastSolutionCollector()
    # Add the decision variables.
    collector.Add(x)
    collector.Add(y)
    # Add the objective.
    collector.AddObjective(obj_expr)
    solver.Solve(decision_builder, [objective, collector])
    if collector.SolutionCount() > 0:
        best_solution = collector.SolutionCount() - 1
        print("Objective value:", collector.ObjectiveValue(best_solution))
        print()
        print('x= ', collector.Value(best_solution, x))
        print('y= ', collector.Value(best_solution, y))

if __name__ == '__main__':
    main()

Objective value: 23

x=  3
y=  2
